In [97]:
require "hdf5"
require "optim"
require "nn"

In [ ]:
--read in files
f = hdf5.open("data.hdf5", "r")
nclasses = f:read("nclasses"):all():long()[1]
nfeatures = f:read("nfeatures"):all():long()[1]

X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
Y_valid = f:read("valid_output"):all()
X_test = f:read("test_input"):all()

In [101]:
X_train = X_train:sub(1, X_train:size(1), 1, 5)
X_valid = X_valid:sub(1,X_valid:size(1), 1, 5)

--X_train = torch.cat(X_train, X_valid, 1)
Y_train = torch.cat(Y_train, Y_valid, 1)

X_test = X_test:sub(1, X_test:size(1), 1, 5)

function changeIndex(X)
    for i=1,5 do
        X:select(2,i):add(1+(i-1)*(37889+2))
    end
    return X
end

X_test = changeIndex(X_test)

In [102]:
function onehotencode(classes, target)
    onehot = torch.zeros(target:size(1), classes)
    for i=1,target:size(1) do
        onehot[i][target[i]] = 1
    end
    return onehot
end

In [103]:
--for now let us disregard additional capitalization features
Y_train_hot = onehotencode(nclasses, Y_train)
--Y_valid_hot = onehotencode(nclasses, Y_valid)

In [104]:
function NB(Y,fc, alpha)
    --calculate log posterior
    local cc = torch.zeros(1, Y:size(2))
    --local fc = torch.zeros(Y:size(2), X:size(2))
    local clp = torch.zeros(1, Y:size(2))

    --collapse column wise so Nx1
    cc:add(Y:sum(1))
    
    --THIS IS ALPHA
    --smoothing
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    print(total)
    print(cc:size())
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local z = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    z[i] = W:index(2, X[i]:long()):sum(2)
    --z[i]:add(b)
  end
  return z
end

function predict_score()
    local c = 0
    for i=1,indices_pred:size(1) do
        if indices_pred[i][1] == indices_true[i][1] then
            c = c + 1
        end
    end
    return c/Y_valid:size()[1]
end

function write2file(fname, pred) 
    f = io.open(fname, "w")
    f:write("ID,Class\n")
    for i=1,pred:size(1) do
        f:write(tostring(i) .. "," .. tostring(pred[i][1]) .. "\n")
    end
    f:close()
end
 

In [105]:
function lookupweights(X, Y)
    W = torch.zeros(45, (37889+2)*5)
    for i=1,X:size(1) do
        for j=1,5 do
            local row = Y[i]
            local column = 1+X[i][j]+(j-1)*(37889+2)
            W[row][column] = W[row][column] + 1
        end
    end
    return W
end

alpha = 1
W = lookupweights(X_train, Y_train)
W, b = NB(Y_train_hot, W, alpha)

 1044474
[torch.DoubleTensor of size 1x1]

  1
 45
[torch.LongStorage of size 2]



In [106]:
predictions = predict(X_test, W, b)
_, indices_pred = torch.max(predictions, 2)
--_, indices_true = torch.max(Y_valid_hot, 2)
--score = predict_score()
--print(score)

In [107]:
write2file("NB2.csv", indices_pred)

In [94]:
X = torch.Tensor{{1,2}}

In [95]:
X2 = torch.Tensor{{1,2}}

In [96]:
torch.cat(X, X2, 1)

 1  2
 1  2
[torch.DoubleTensor of size 2x2]

